In [1]:
from datasets import load_dataset
from transformers import pipeline
import torch

torch.set_grad_enabled(False)

[2023-08-26 23:06:32,622] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [2]:
device = "cuda:0"

In [3]:
dataset = load_dataset("SetFit/emotion", split="train")

Found cached dataset json (/home/heegyu/.cache/huggingface/datasets/SetFit___json/SetFit--emotion-d091a62525eb1bd7/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)


In [4]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification


toxicity_model_id = "facebook/roberta-hate-speech-dynabench-r4-target"
toxicity_tokenizer = RobertaTokenizer.from_pretrained(toxicity_model_id)
# We load the toxicity model in fp16 to save memory.
toxicity_model = RobertaForSequenceClassification.from_pretrained(toxicity_model_id, torch_dtype=torch.float16).eval().to(
    device
)

In [5]:
def is_toxic(x):
    texts = x["text"]
    inputs = toxicity_tokenizer(texts, return_tensors="pt", padding=True, truncation=True).to(device)
    logits = toxicity_model(**inputs).logits.softmax(-1)
    toxicity_labels = (logits[:, 1]).tolist()
    return {
        "toxicity": toxicity_labels
    }

In [6]:
toxified = dataset.map(is_toxic, batched=True, batch_size=8)

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

In [7]:
df = toxified.to_pandas()
df

,text,label,label_text,toxicity
0,i didnt feel humiliated,0,sadness,0.000217
1,i can go from feeling so hopeless to so damned...,0,sadness,0.002481
2,im grabbing a minute to post i feel greedy wrong,3,anger,0.005100
3,i am ever feeling nostalgic about the fireplac...,2,love,0.000143
4,i am feeling grouchy,3,anger,0.000436
...,...,...,...,...
15995,i just had a very brief time in the beanbag an...,0,sadness,0.011353
15996,i am now turning and i feel pathetic that i am...,0,sadness,0.020782
15997,i feel strong and good overall,1,joy,0.000151
15998,i feel like this was such a rude comment and i...,3,anger,0.001461


In [8]:
len(df[df.toxicity > 0.1].text.tolist())

782

In [9]:
df.to_csv("emotion_toxic.csv")